# Baseline Policy

In [1]:
using Revise
using Random
using Printf
using DataStructures
using Flux
using POMDPs
using POMDPModelTools
using POMDPSimulators
using BeliefUpdaters
using POMDPPolicies
using DiscreteValueIteration
using MDPModelChecking
using StaticArrays
using RLInterface
using DeepQLearning
using AutomotiveDrivingModels
using AutomotivePOMDPs
using AutomotiveSensors
using LocalApproximationValueIteration
using Reel
using AutoViz
using ProgressMeter
using JLD2
using FileIO
using BSON
using PedCar

loaded


┌ Info: Recompiling stale cache file /mnt/c/Users/Maxime/wsl/.julia/compiled/v1.0/LocalApproximationValueIteration/Dvh7I.ji for LocalApproximationValueIteration [a40420fb-f401-52da-a663-f502e5b95060]
└ @ Base loading.jl:1184
┌ Warning: Package LocalApproximationValueIteration does not have Random in its dependencies:
│ - If you have LocalApproximationValueIteration checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LocalApproximationValueIteration
└ Loading Random into LocalApproximationValueIteration from project dependency, future warnings for LocalApproximationValueIteration are suppressed.
┌ Info: Recompiling stale cache file /mnt/c/Users/Maxime/wsl/.julia/compiled/v1.0/PedCar/NmDDZ.ji for PedCar [90cf7f26-d5c7-593d-a0e1-4a8367407571]
└ @ Base loading.jl:1184
┌ Warning: Package PedCar does not have AutomotivePOMDPs in its depen

In [2]:
includet("../src/masking.jl")
includet("../src/masked_dqn.jl")
includet("../src/qmdp_approximation.jl")
includet("../src/baseline_policy.jl")
includet("../src/decomposed_tracking.jl")
includet("../src/decomposition.jl")
includet("../src/util.jl")
includet("../src/render_helpers.jl")

In [3]:
rng = MersenneTwister(1);
cam = FitToContentCamera(0.);

## Scenario

In [4]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =[VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

In [5]:
mdp = PedCarMDP(env=env, pos_res=2.0, vel_res=2., ped_birth=0.7, car_birth=0.7);
init_transition!(mdp);

In [6]:
pomdp = UrbanPOMDP(env=env,
                   sensor = PerfectSensor(),
                   ego_goal = LaneTag(2, 1),
                   max_cars=1, 
                   max_peds=1, 
                   car_birth=0.7, 
                   ped_birth=0.7, 
                   max_obstacles=0., # no fixed obstacles
                   lidar=false,
                   ego_start=20,
                   ΔT=0.1);

## Initialize Baseline Policy

Input: a driving scene, use `obs_to_scene` to convert the observation vector

In [7]:
@load "../pc_util_processed_low.jld2" qmat util pol
safe_policy = ValueIterationPolicy(mdp, qmat, util, pol);
threshold = 0.99
mask = SafetyMask(mdp, safe_policy, threshold);

In [8]:
mutable struct MaskedEgoBaseline <: Policy
    pomdp::UrbanPOMDP
    model::DriverModel
    mask::SafetyMask
    sa::Vector{UrbanAction}
end

In [14]:
function POMDPs.action(policy::MaskedEgoBaseline, o::UrbanObs)
    policy.sa = safe_actions(policy.pomdp, policy.mask, o)
    s = obs_to_scene(policy.pomdp, o)
    observe!(policy.model, s, policy.pomdp.env.roadway, EGO_ID)
    acts = [a.acc for a in ordered_actions(pomdp)]
    ai = argmin(abs.(policy.model.a.a_lon .- acts))
    a = ordered_actions(pomdp)[ai]
    if a ∈ policy.sa
        return a 
    else 
        ai = argmax([a.acc for a in policy.sa])
        return policy.sa[ai]
    end
end

function POMDPModelTools.action_info(policy::MaskedEgoBaseline, o::UrbanObs)
    a = action(policy, o)
    return a, (policy.model.a.a_lon, policy.sa)
end

In [68]:
ego_model = get_ego_baseline_model(env);

In [69]:
masked_baseline = MaskedEgoBaseline(pomdp, ego_model, mask, UrbanAction[]);
baseline = EgoBaseline(pomdp, ego_model);

## Evaluation

In [112]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, baseline, n_ep=1000, max_steps=400, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

Progress: 100%|█████████████████████████████████████████|  ETA: 0:00:01

506.502350 seconds (434.04 M allocations: 35.950 GiB, 7.41% gc time)
Summary for 1000 episodes: 
Average reward: 0.004 
Average # of steps: 251.184 
Average # of violations: 0.000 


Progress: 100%|█████████████████████████████████████████| Time: 0:08:26


## Visualize Baseline Policy

In [77]:
function POMDPModelTools.generate_sori(pomdp::UrbanPOMDP, s::Scene, a::UrbanAction, rng::AbstractRNG)
    sp, o, r = generate_sor(pomdp, s, a, rng)
    return sp, o, r, deepcopy(pomdp.models)
end

In [113]:
hr = HistoryRecorder(rng=rng, max_steps=400)
s0 = initialstate(pomdp, rng)
up = PreviousObservationUpdater()
o0 = generate_o(pomdp, s0, UrbanAction(0.), s0, rng)
b0 = initialize_belief(up, o0)
@time hist = simulate(hr, pomdp, baseline, up, b0, s0);

  0.567586 seconds (677.19 k allocations: 57.323 MiB, 5.01% gc time)


In [111]:
function find_collision(pomdp, test_policy, rng)
    up = PreviousObservationUpdater()
    @showprogress for ep=1:10000
        hr = HistoryRecorder(rng=rng, max_steps=400)
        s0 = initialstate(pomdp, rng)
        o0 = generate_o(pomdp, s0, UrbanAction(0.), s0, rng)
        b0 = initialize_belief(up, o0)
        hist2 = simulate(hr, pomdp, test_policy, up, b0, s0)
        if sum(hist2.reward_hist .< 0.) != 0.
            println("Crash")
            return hist2
        end
    end
    return hist2 
end
hist = find_collision(pomdp, baseline, rng);

Progress: 100%|█████████████████████████████████████████| Time: 2:36:10


UndefVarError: UndefVarError: hist2 not defined

In [114]:
animate_history(hist, pomdp,
                action_overlays = a -> [TextOverlay(text = ["Acc: $(a.acc) m/s^2"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3., 8.), incameraframe=true)],                                   
                ainfo_overlays = ai -> [TextOverlay(text = ["Acc: $(ai[1]) m/s^2"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3., 6.), incameraframe=true)],
#                                         TextOverlay(text = ["Available Actions: $([a.acc for a in ai[2]])"], font_size=20,pos=VecE2(pomdp.env.params.x_min + 3.,10.), incameraframe=true)],
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                cam =  StaticCamera(VecE2(0., -8.), 12.0),
                speed_factor=2)

Frames{MIME{Symbol("image/png")}}("/tmp/tmpGSbpdj", 0x0000000000000190, 20.0, nothing)

In [101]:
step = 44
s = hist.state_hist[step+1]
model = hist.ainfo_hist[step][2]
models = hist.info_hist[step]
vehid = 101
vehm = models[vehid]
cwm = model.crosswalk_drivers[3]
intm = model.intersection_driver

TTCIntersectionDriver
  a: LonAccelDirection
  navigator: RouteFollowingIDM
  intersection: Array{Lane}((2,))
  intersection_pos: VecSE2{Float64}
  ttc_threshold: Float64 3.0
  horizon: Float64 20.0
  stop_delta: Float64 4.0
  accel_tol: Float64 0.0
  priorities: Dict{Tuple{LaneTag,LaneTag},Bool}
  priority: Bool false
  stop: Bool false


In [109]:
m = deepcopy(vehm)
observe!(m, s, pomdp.env.roadway, 101)
m.a

ConstantSpeedDawdling(1.0, 0.0)

In [107]:
vehm.conflict_lanes

5-element Array{Lane,1}:
 Lane(LaneTag(5, 1), CurvePt[CurvePt({-1.500, -4.000, -1.571}, 0.000, NaN, NaN), CurvePt({-1.500, -30.000, -1.571}, 26.000, NaN, NaN)], 3.0, SpeedLimit(-Inf, Inf), LaneBoundary(:solid, :white), LaneBoundary(:solid, :white), LaneConnection[], LaneConnection[LaneConnection(U, CurveIndex(1, 0.000), RoadIndex({24, 1.000000}, {12, 1}), LaneConnection(U, CurveIndex(1, 0.000), RoadIndex({24, 1.000000}, {10, 1})])
 Lane(LaneTag(6, 1), CurvePt[CurvePt({1.500, -30.000, 1.571}, 0.000, NaN, NaN), CurvePt({1.500, -4.000, 1.571}, 26.000, NaN, NaN)], 3.0, SpeedLimit(-Inf, Inf), LaneBoundary(:solid, :white), LaneBoundary(:solid, :white), LaneConnection[LaneConnection(D, CurveIndex(1, 1.000), RoadIndex({1, 0.000000}, {15, 1}), LaneConnection(D, CurveIndex(1, 1.000), RoadIndex({1, 0.000000}, {13, 1})], LaneConnection[])      
 Lane(LaneTag(17, 1), CurvePt[CurvePt({6.000, -7.000, 1.571}, 0.000, 0.000, NaN), CurvePt({6.000, 7.000, 1.571}, 14.000, 0.000, NaN)], 4.0, SpeedLimit(-Inf

In [98]:
ped = s[findfirst(101, s)]
cos(ped.state.posF.ϕ)
ped.state.posF

Frenet(RoadIndex({1, 0.718562}, {19, 1}), 10.060, -0.530, 3.142)

In [41]:
cwm.yield

false

In [43]:
get_conflict_lanes(env.crosswalks[3], env.roadway)

5-element Array{Lane,1}:
 Lane(LaneTag(5, 1), CurvePt[CurvePt({-1.500, -4.000, -1.571}, 0.000, NaN, NaN), CurvePt({-1.500, -30.000, -1.571}, 26.000, NaN, NaN)], 3.0, SpeedLimit(-Inf, Inf), LaneBoundary(:solid, :white), LaneBoundary(:solid, :white), LaneConnection[], LaneConnection[LaneConnection(U, CurveIndex(1, 0.000), RoadIndex({24, 1.000000}, {12, 1}), LaneConnection(U, CurveIndex(1, 0.000), RoadIndex({24, 1.000000}, {10, 1})])
 Lane(LaneTag(6, 1), CurvePt[CurvePt({1.500, -30.000, 1.571}, 0.000, NaN, NaN), CurvePt({1.500, -4.000, 1.571}, 26.000, NaN, NaN)], 3.0, SpeedLimit(-Inf, Inf), LaneBoundary(:solid, :white), LaneBoundary(:solid, :white), LaneConnection[LaneConnection(D, CurveIndex(1, 1.000), RoadIndex({1, 0.000000}, {15, 1}), LaneConnection(D, CurveIndex(1, 1.000), RoadIndex({1, 0.000000}, {13, 1})], LaneConnection[])      
 Lane(LaneTag(17, 1), CurvePt[CurvePt({6.000, -7.000, 1.571}, 0.000, 0.000, NaN), CurvePt({6.000, 7.000, 1.571}, 14.000, 0.000, NaN)], 4.0, SpeedLimit(-Inf